## Uploads data sets

In [1]:
# libs
import pandas as pd

In [40]:
# Vendas
df_vendas = pd.read_parquet('fato_vendas_varejo.parquet')

# Filias

df_filiais = pd.read_parquet('filiais.parquet')

# Metas Vendas

df_metas_vendas = pd.read_parquet('metas_vendas_varejo.parquet')


### Metas Vendas

In [41]:
df_metas_vendas.head(10)

,codigo_filial,data_meta_venda,meta_n_med
0,1740,2025-01-01 03:00:00.000,4652.32
1,1830,2025-01-01 03:00:00.000,7870.85
2,1524,2025-01-01 03:00:00.000,6144.89
3,1677,2025-01-01 03:00:00.000,11840.37
4,1758,2025-01-01 03:00:00.000,9554.39
5,1656,2025-01-01 03:00:00.000,0.00
6,1581,2025-01-01 03:00:00.000,1680.03
7,1812,2025-01-01 03:00:00.000,8568.37
8,1746,2025-01-01 03:00:00.000,9320.65
9,1635,2025-01-01 03:00:00.000,7079.00


In [9]:
print("\n" + "="*60)
print("METAS shape:", df_metas_vendas.shape)

print("\n[METAS] missing por coluna (qtd):")
na_metas = df_metas_vendas.isna().sum().sort_values(ascending=False)
print(na_metas[na_metas > 0])

print("\n[METAS] duplicadas (linha inteira):", df_metas_vendas.duplicated().sum())

print("\n[METAS] duplicadas por (codigo_filial, data_meta_venda):",
      df_metas_vendas.duplicated(subset=["codigo_filial", "data_meta_venda"]).sum())


METAS shape: (42896, 3)

[METAS] missing por coluna (qtd):
Series([], dtype: int64)

[METAS] duplicadas (linha inteira): 0

[METAS] duplicadas por (codigo_filial, data_meta_venda): 0


### Informações Filiais

In [17]:
df_filiais.head(10)

,codigo_filial,faixa_vida,localidade,uf,tipo_estabelecimento,delivery,metragem_area_venda,panvel_clinic,estacionamento,atendimento_24_horas
0,1761,MAIS DE 3 ANOS,APUCARANA,PR,BAIRRO,SIM,503.5618,SIM,SIM,NÃO
1,1740,MAIS DE 3 ANOS,LONDRINA,PR,CENTRO,NÃO,531.0054,SIM,SIM,NÃO
2,1689,MAIS DE 3 ANOS,LONDRINA,PR,BAIRRO,NÃO,521.7424,NÃO,SIM,NÃO
3,1611,MAIS DE 3 ANOS,LONDRINA,PR,BAIRRO,NÃO,565.2000,SIM,SIM,NÃO
4,1764,MAIS DE 3 ANOS,LONDRINA,PR,CENTRO,NÃO,572.1708,SIM,SIM,NÃO
5,1884,MENOS DE 1 ANO,ARAPONGAS,PR,MALL,NÃO,471.2826,SIM,SIM,NÃO
6,1869,MENOS DE 1 ANO,TELEMACO BORBA,PR,CENTRO,SIM,566.5188,SIM,SIM,NÃO
7,1803,ENTRE 2-3 ANOS,CAMBE,PR,CENTRO,SIM,531.0682,SIM,SIM,NÃO
8,1782,MAIS DE 3 ANOS,ARAPONGAS,PR,CENTRO,SIM,585.9240,SIM,SIM,NÃO
9,1767,MAIS DE 3 ANOS,LONDRINA,PR,BAIRRO,SIM,626.6812,SIM,SIM,NÃO


In [11]:
print("\n" + "="*60)
print("FILIAIS shape:", df_filiais.shape)

print("\n[FILIAIS] missing por coluna (qtd):")
na_filiais = df_filiais.isna().sum().sort_values(ascending=False)
print(na_filiais[na_filiais > 0])

print("\n[FILIAIS] duplicadas (linha inteira):", df_filiais.duplicated().sum())

print("\n[FILIAIS] duplicadas por codigo_filial:", 
      df_filiais.duplicated(subset=["codigo_filial"]).sum())


FILIAIS shape: (124, 10)

[FILIAIS] missing por coluna (qtd):
Series([], dtype: int64)

[FILIAIS] duplicadas (linha inteira): 0

[FILIAIS] duplicadas por codigo_filial: 0


### Vendas

In [ ]:
df_vendas.tail()

,codigo_documento_saida,codigo_filial,data_emissao,quantidade,faturamento
5533166,824239,1542,2025-09-22 00:00:00.000,3.0,47.08359
5533167,1343474,1551,2025-09-08 00:00:00.000,6.0,106.73118
5533168,823095,1698,2025-09-06 00:00:00.000,3.0,47.08359
5533169,593391,1641,2025-09-28 00:00:00.000,6.0,62.75718
5533170,4692,1881,2025-09-22 00:00:00.000,6.0,25.06518
...,...,...,...,...,...
5534161,100005,1803,2025-03-12 00:00:00.000,3.0,26.66709
5534162,548192,1641,2025-03-02 00:00:00.000,3.0,73.78209
5534163,10791,1842,2025-03-30 00:00:00.000,6.0,9.98838
5534164,667263,1596,2025-03-02 00:00:00.000,6.0,37.62918


In [21]:
print("VENDAS shape:", df_vendas.shape)

print("\n[VENDAS] missing por coluna (qtd):")
na_vendas = df_vendas.isna().sum().sort_values(ascending=False)
print(na_vendas[na_vendas > 0])

print("\n[VENDAS] duplicadas (linha inteira):", df_vendas.duplicated().sum())

print("\n[VENDAS] duplicadas por codigo_documento_saida:", 
      df_vendas.duplicated(subset=["codigo_documento_saida"]).sum())

print("\n[VENDAS] quantas linhas por (codigo_filial, data_emissao) — ajuda a ver se é 1/dia ou várias vendas/dia:")
print(df_vendas.groupby(["codigo_filial", "data_emissao"]).size().describe())



VENDAS shape: (5534166, 5)

[VENDAS] missing por coluna (qtd):
Series([], dtype: int64)

[VENDAS] duplicadas (linha inteira): 0

[VENDAS] duplicadas por codigo_documento_saida: 3778312

[VENDAS] quantas linhas por (codigo_filial, data_emissao) — ajuda a ver se é 1/dia ou várias vendas/dia:
count    42420.000000
mean       130.461245
std         67.060052
min          1.000000
25%         91.000000
50%        117.000000
75%        150.000000
max       1144.000000
dtype: float64


In [33]:
loja = 1614

# --- FILTROS ---
v = df_vendas[df_vendas["codigo_filial"] == loja].copy()
m = df_metas_vendas[df_metas_vendas["codigo_filial"] == loja].copy()

In [34]:
v["data_emissao"] = pd.to_datetime(v["data_emissao"])
v = v.sort_values(["data_emissao", "codigo_documento_saida"]).reset_index(drop=True)

v.head(20)

,codigo_documento_saida,codigo_filial,data_emissao,quantidade,faturamento
0,925182,1614,2025-01-01,15.0,81.50895
1,925183,1614,2025-01-01,36.0,516.69450
2,925184,1614,2025-01-01,6.0,125.29449
3,925185,1614,2025-01-01,6.0,22.55238
4,925187,1614,2025-01-01,6.0,151.96158
5,925188,1614,2025-01-01,3.0,153.59490
6,925189,1614,2025-01-01,6.0,61.50078
7,925190,1614,2025-01-01,21.0,1434.65175
8,925191,1614,2025-01-01,3.0,386.02890
9,925193,1614,2025-01-01,3.0,248.10759


In [35]:
print("linhas:", len(v))
print("documentos únicos:", v["codigo_documento_saida"].nunique())
print("duplicados (por documento):", v.duplicated(subset=["codigo_documento_saida"]).sum())

linhas: 85804
documentos únicos: 85804
duplicados (por documento): 0


In [36]:
m

,codigo_filial,data_meta_venda,meta_n_med
53,1614,2025-01-01,23642.44
208,1614,2025-01-02,36941.32
237,1614,2025-01-03,36941.32
416,1614,2025-01-04,36941.32
511,1614,2025-01-05,29553.05
...,...,...,...
42378,1614,2025-12-27,54645.99
42474,1614,2025-12-28,38562.25
42628,1614,2025-12-29,75574.24
42730,1614,2025-12-30,76930.69
